In [6]:
import numpy as np
import math
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

#Key Matrix Generator
def keyGen(key):
    if key:
        key = key.split()
        key = list(map(int, key)) 
        n = int(math.sqrt(len(key)))
        keyMat = np.array(key).reshape(n,n)
        #print(keyMat)

    return keyMat

#form dictionary of chars a-z and map them to 0-25 numbers
def charsDictGen():    
    charsDict = {"a":None}
    init = 'a'
    for i in range(0,26):
        charsDict[init] = i
        init = chr(ord(init) + 1)
    return charsDict

#form dictionary of 0-25 numbers and map them to chars a-z 
def numbersDictGen():    
    numsDict = {0:None}
    init = 'a'
    for i in range(0,26):
        numsDict[i] = init
        init = chr(ord(init) + 1)
    return numsDict

#create matrix of plain text of such that it creates linear combinations of vectors of size n (nxn is key size)
def createMatrixOfText(text, n):
    charDict = charsDictGen()
    rem = len(text)%n
    
    if rem != 0:
        xToAppend = n - rem
        
        for i in range(xToAppend):
            text += "z"
    
    quotient = int(len(text)/n)
    textMat = np.zeros((quotient, n), dtype=int)
    
    count = 0
    for m in range(quotient):
        for p in range(n):
            if text[count] != ' ':
                textMat[m][p] = charDict[text[count].lower()]
                count += 1
    #print(textMat.transpose())
    return textMat.transpose()

#Re insert spaces in text to make is readable
def reInsertSpaces(initialText, finalText):
    output = ""
    count = 0
    for i in initialText:
        if i == ' ':
            output += ' '
        else:
            output += finalText[count]
            count += 1
    return output

#calculate modulo inverse of key matrix
def calcModInverse(mat):    
    det = np.linalg.det(mat)
    #print(round(det))
    cofactorMat = np.linalg.inv(mat) * det
    #cofactorMat = cofactorMat.astype(int)
    detMod = calcMultiplicativeInverse(round(det))
    invMat = cofactorMat * detMod
    invMat = np.mod(invMat, 26)

    #print(invMat)
    #print("\n Cofactor Matrix Transpose: \n", cofactorMat, "\n", "\n", invMat, "\n", det, "\n", detMod)
    return invMat.round()
    

#multiplicative inverse of determent mod(26)
def calcMultiplicativeInverse(determinant):
    multi_inverse = -1
    for i in range(26):
        inv = determinant * i
        if inv % 26 == 1:
            multi_inverse = i
            break
    return multi_inverse

#check if key is nxn or not
def isPerfectSquare(k):
    k = k.split()
    k = list(map(int, k))
    sr = math.sqrt(len(k))    
    # If square root is an integer 
    return ((sr - math.floor(sr)) == 0) 

def isInvertible(mat):
    det = np.linalg.det(mat)
    if (int(round(det))!= 0) and (math.gcd(int(round(det)), 26)== 1):
        return True
    else:
        return False

#Encrypt the plain text with key using Hill Cipher Algorithm
def hill_encrypt(pText, cKey):
    matPText = createMatrixOfText(pText.replace(' ',''),len(cKey[0]))
    matCText = np.zeros((matPText.shape), dtype = int)
    for i in range(len(matPText[0])):
        matCText[:,i] = cKey.dot(matPText[:,i])
    
    matCText = matCText.transpose()
    cText = ""
    numToChar = numbersDictGen()
    for row in matCText:
        for element in row:
            element = element%26
            cText += numToChar[element]
    cText = reInsertSpaces(pText, cText)
    print("Encrypted Text: \n", cText)

#Decrypt the cipher text for a given key using Hill Cipher Algorithm
def hill_decrypt(cText, dKey):
    matCText = createMatrixOfText(cText.replace(' ',''),len(dKey[0]))
    #print(matCText)
    matPText = np.zeros((matCText.shape), dtype = int)
    dKey = calcModInverse(dKey)
    #print("dKey : {}", dKey)
    for i in range(len(matCText[0])):
        matPText[:,i] = dKey.dot(matCText[:,i])

    matPText = matPText.transpose()
    pText = ""
    numToChar = numbersDictGen()
    for row in matPText:
        for element in row:
            element = element%26
            pText += numToChar[element]
    pText = reInsertSpaces(cText, pText)
    print("Decrypted Text: \n", pText)
    

#init Main() program
if __name__ == '__main__':
    opType = input("Choose task for Hill Cipher. : \n Encrypt: Enter 1. \n Decrypt: Enter 2. \n ")

    if opType == '1':
        #Get plain text
        plainText = input("Enter Plain Text (Only a-z characters): \n")
        #Get Key
        ciKey = input("Enter cipher key(space separated): (eg. for 2x2: 22 3 9 6) \n")
        ps = isPerfectSquare(ciKey)
        if ps:        
            ciKey = keyGen(ciKey)
            isInv = isInvertible(ciKey)
            if isInv:
                encryptedText = hill_encrypt(plainText, ciKey)
            else:
                print("ALERT : This matrix is either singular or non-invertible for mod 26")
        else: 
            print("ALERT : Key is not nxn. Kindly enter a square matrix")
    elif opType == '2':
        #Get plain text
        cipherText = input("Enter Cipher Text (Only a-z characters): \n")
        #Get Key
        ciKey = input("Enter decipher key(space separated): (eg. for 2x2: 22 3 9 6) \n")
        ps = isPerfectSquare(ciKey)
        if ps:
            ciKey = keyGen(ciKey)
            isInv = isInvertible(ciKey)
            if isInv:
                decryptedText = hill_decrypt(cipherText, ciKey)
            else:
                print("ALERT : This matrix is either singular or non-invertible for mod 26")
        else: 
            print("ALERT : Key is not nxn. Kindly enter a square matrix")
    else:
        print("ALERT : Please choose a valid option.")

Choose task for Hill Cipher. : 
 Encrypt: Enter 1. 
 Decrypt: Enter 2. 
 2
Enter Cipher Text (Only a-z characters): 
zskkkisx la nzkkgi hu p emexwfkfqm oxkse xfxk nukycj cpmdhuc cmpdi zee cxqgma lf xkj gronkxkzxkse xfy mitono fwiuj lqwvikhus iq tml gfel eoilphnav busfx xffk nzkkgi ylgf na eoilphnab ndkp up iivgea nqcitsst zdkzeagg gldji hoi v grihkxk zcigswd wy wzurtc jw xfn qcewy la jwmo eoskhufkkaf xkjse eoskhufkkacr zyk k wvownf la jwmo frwi xfemxset lgage xna ih wvivont lf owvkug gp ywvkuhuvoadsdazg g qbxcxwa ql dkj wvho gwrlw rgzhsjihuh mwhsxu iwhni bgkjyzwyu emedv qtx flwn qwitzjwazwvw rilj fkw sv anjvkytc nav peozetc frwiehu ke peeayyjsx ze eek mwid cqwtamol qc ijwecazp hkjilikxabizey ol dxwyjsx sv nzkkgi sv gwof xfadqid vkj ilgralsso hjwmxaljwl qfouo eza lpcvuhuh mkjeenfogpukju rmedv vgxk ih ukwv wvwautik ggkm xfk kdiorhR
Enter decipher key(space separated): (eg. for 2x2: 22 3 9 6) 
22 3 9 6
Decrypted Text: 
 speaking of dreams in a figurative sense then slowly talking about it

In [15]:
1 2 19 11 5 22 3 10 4 10 11 21 13 3 15 10

array([[22,  3],
       [ 9,  6]])

In [ ]:
6 24 1 13 16 10 20 17 15
25 9
13 2